In [1]:
# add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

# #sklearnex
# from sklearnex import patch_sklearn
# patch_sklearn()
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# StratifiedKFold
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import (accuracy_score,
                            recall_score,
                            precision_score,
                            f1_score,
                            roc_auc_score,
                            roc_curve,
                            matthews_corrcoef,
)
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
import scipy
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
import cv2 as cv
from radiomics import featureextractor
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.preprocessing import StandardScaler
# RandomizedSearchCV
#Import paths and patients classes
from notebooks.info import path_label, patient
import notebooks.utils as utils

def get_features(show=False):
    """returns available features for the uncertainty scheme

    Args:
        show (bool, optional): if information about the excluded features and nxp size should be given. Defaults to False.

    Returns:
        pd.DataFrame: features loaded
    """
    budget_path = repo_path/ 'data' / 'budget' / 'budget_ROI_and_rad.csv'
    excluded, _ = utils.get_ex_included(budget_path) # get excluded features due to their budget value
    features = pd.read_csv(repo_path / 'data' / 'features' / f'feat_vector.csv', index_col=0)
    # remove features in excluded list
    features = features.drop(excluded, axis=1)
    if show:
        print(f'The features removed are {excluded.values} beacuse their budget value is greater than 1')
        n = features.shape[0] # number of patients
        p = features.shape[1] # number of features
        print(f'The number of patients (n) is: {n}\nThe number of features (p) is: {p}')
    return features



In [2]:

# experiment HP
label = 'RP' # receptor type (RP, RE, ki67)

# get pat info and features
info = path_label()
features = get_features(show=True)

The features removed are ['glcm_ClusterShade'] beacuse their budget value is greater than 1
The number of patients (n) is: 33
The number of features (p) is: 101


In [16]:
y = np.asarray(info.labels_list(label))
x = np.asarray(features)
x[:,-1]

array([0.00068904, 0.00056375, 0.00093332, 0.00063497, 0.0008432 ,
       0.0003523 , 0.00084584, 0.00053956, 0.0005096 , 0.00049393,
       0.00066288, 0.00086068, 0.00082091, 0.00043792, 0.00058226,
       0.00055402, 0.0005179 , 0.00064137, 0.00079032, 0.00084167,
       0.00082123, 0.00082932, 0.00063231, 0.00050516, 0.00073148,
       0.0006744 , 0.00057246, 0.0004898 , 0.00031879, 0.00059651,
       0.00044299, 0.00097484, 0.00054212])